In [ ]:
import deeppavlov
import json
#import numpy as np
import tensorflow as tf

#from itertools import chain
#from pathlib import Path

In [ ]:
from deeppavlov.core.commands.train import build_model_from_config
from deeppavlov.core.data.dataset_reader import DatasetReader
#from deeppavlov.core.data.utils import download_decompress
from deeppavlov.core.common.registry import register

In [ ]:
@register('char_lm_dataset_reader')
class CharLmDatasetReader(DatasetReader):
    """
    Dataset reader which reads 
    """
    def read(self, dir_path: str, mode='self_original'):
        dir_path = Path(dir_path)
        dataset = {}
        for dt in ['train', 'valid', 'test']:
            dataset[dt] = self._parse_data(dir_path / '{}_{}.txt'.format(dt, mode))

        return dataset

    @staticmethod
    def _parse_data(filename):
        examples = []
        print(filename)
        curr_persona = []
        curr_dialog_history = []
        persona_done = False
        with filename.open('r') as fin:
            for line in fin:
                line = ' '.join(line.strip().split(' ')[1:])
                your_persona_pref = 'your persona: '
                if line[:len(your_persona_pref)] == your_persona_pref and persona_done:
                    curr_persona = [line[len(your_persona_pref):]]
                    curr_dialog_history = []
                    persona_done = False
                elif line[:len(your_persona_pref)] == your_persona_pref:
                    curr_persona.append(line[len(your_persona_pref):])
                else:
                    persona_done = True
                    x, y, _, candidates = line.split('\t')
                    candidates = candidates.split('|')
                    example = {
                        'persona': curr_persona,
                        'x': x,
                        'y': y,
                        'dialog_history': curr_dialog_history[:],
                        'candidates': candidates,
                        'y_idx': candidates.index(y)
                    }
                    curr_dialog_history.extend([x, y])
                    examples.append(example)

        return examples

In [1]:
print(len('DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'))

85


In [4]:
a = 'a' / 'b'
print(a)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [7]:
def greeting(name: str) -> str:
    return name

print(greeting(1))

1


In [8]:
a, b = (1, 2)

In [9]:
print(a)

1


In [6]:
print(0 or 3)

3


In [7]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer

tokenizer = RussianTokenizer(ngram_range=[1, 1])
batch = ['abc', 'def']
print(tokenizer(batch))

2018-08-01 15:29:52.751 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/anton/dpenv/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-08-01 15:29:52.805 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


[['abc'], ['def']]


In [9]:
from deeppavlov.models.tokenizers.utils import ngramize

items = ['abcd', 'efgh']
for i in ngramize(items, ngram_range=(2, 2)):
    print(i)

['abcd efgh']


In [17]:
def f(a, b, c='3'):
    """Function test its fields"""
    d = a + b + c
    return d

print(f)
print(f.__call__.__call__.__call__.__call__.__call__.__name__)
print(f.__name__)
print(f.__doc__)
print(f.__defaults__)
print(f.__kwdefaults__)
print(f.__code__)

<function f at 0x7ff83c3356a8>
__call__
f
Function test its fields
('3',)
None
<code object f at 0x7ff83c326e40, file "<ipython-input-17-3e15ce66597b>", line 1>


In [23]:
def f():
    print(f)
f()

<function f at 0x7ff83c335e18>


In [28]:
x = input("Do you need the answer? (y/n): ")
if x.lower() == "y":
    required = True
else:
    required = False
    
def the_answer(self, *args):              
        return 42
    
class EssentialAnswers(type):
    
    def __init__(cls, clsname, superclasses, attributedict):
        if required:
            cls.the_answer = the_answer
                           
    
class Philosopher1(metaclass=EssentialAnswers): 
    pass


print(Philosopher1.the_answer)
class Philosopher2(metaclass=EssentialAnswers): 
    pass
class Philosopher3(metaclass=EssentialAnswers): 
    pass
    
    
plato = Philosopher1()
print(plato.the_answer())
kant = Philosopher2()
# let's see what Kant has to say :-)
print(kant.the_answer())

Do you need the answer? (y/n): y


AttributeError: type object 'EssentialAnswers' has no attribute 'the_answer'

In [29]:
from char_lm_vocab import CharLMVocabulary
vocab = CharLMVocabulary(
    save_path='./vocab.dict',
    load_path='./vocab.dict',
)
text = 'DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'
vocab.fit(text)
vocab.save()


2018-08-03 15:36:34.385 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.400 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.412 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 59: [saving vocabulary to /home/anton/DeepPavlov/download/vocab.dict]


In [31]:
print(len(vocab))
print(vocab[5])
print('z' in vocab)
print('a' in vocab)
print(vocab.is_str_batch([]))
print(vocab.is_str_batch(['c']))
print(vocab(['a', 'b', 'я']))

27
r
False
True
False
True
[3, 14, None]


In [1]:
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as LSTM
import numpy as np

num_units = 100
voc_size = 20
inps = tf.placeholder(tf.int32, shape=[None, None])
lbls = tf.placeholder(tf.int32, shape=[None, None])
prep_inps = tf.one_hot(inps, voc_size)
prep_lbls = tf.one_hot(lbls, voc_size)

out_w = tf.Variable(
    tf.truncated_normal([num_units, voc_size])
)

with tf.device('/gpu:0'):
    lstm = LSTM(
        2,
        num_units,
        input_mode='skip_input',
    )

bs = tf.shape(inps)[1:-1]
state_shape = tf.concat([[2], bs, [num_units]], 0)
with tf.device('/gpu:0'):
    lstm_res, state = lstm(
        prep_inps
    )
    print(lstm_res)
    logits = tf.matmul(tf.reshape(lstm_res, [-1, num_units]), out_w)
    predictions = tf.nn.softmax(logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits,
        labels=tf.reshape(prep_lbls, [-1, voc_size])
    ))
    opt = tf.train.GradientDescentOptimizer(1.)
    train_op = opt.minimize(loss)
    
nu = 10
bs = 32
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        inputs = np.random.randint(0, voc_size, size=(nu, bs))
        labels = np.random.randint(0, voc_size, size=(nu, bs))
        _, preds = sess.run([train_op, preds], feed_dict={inps: inputs, lbls: labels})

    


KeyboardInterrupt: 

In [3]:
import itertools

def create_distribute_map(num_distributed, result_length):
    quotient = result_length / num_distributed
    num_filled = 0
    if num_distributed < result_length:
        num_repeats = list()
        for i in range(1, num_distributed):
            filled_space = quotient * i
            print(filled_space)
            truncated = int(filled_space)
            num_repeats.append(truncated - num_filled)
            num_filled = truncated
        num_repeats.append(result_length - num_filled)
    else:
        num_repeats = [1] * result_length
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(num_repeats)]))
    return map_

print(create_distribute_map(3, 2))

[0, 1]


In [6]:
for idx in range(10, -1, -1):
    print(idx)

10
9
8
7
6
5
4
3
2
1
0


In [17]:
with open('debug.txt', 'r') as f:
    t = f.read()
print(t.split('\n')[0])
print('"%s"' % t)
l = []
print(l[0])


""


IndexError: list index out of range

In [7]:
import itertools
import copy
from useful_functions import all_combs

def create_even_distribution(num_unique_items, num_slots):
    """Returns a list with number of slots taken by each item. If num_slots < num_unique_items not all items
    will be distributed"""
    items_per_slot_float = num_slots / num_unique_items
    total_slots_filled_saved = 0
    if num_unique_items < num_slots:
        item_distribution = list()
        for i in range(1, num_unique_items):
            total_slots_float = items_per_slot_float * i
            total_slots_filled = int(total_slots_float)
            item_distribution.append(total_slots_filled - total_slots_filled_saved)
            total_slots_filled_saved = total_slots_filled
        item_distribution.append(num_slots - total_slots_filled_saved)
    else:
        item_distribution = [1] * num_slots
    return item_distribution


def create_distribute_map(num_unique_items, num_slots):
    item_distribution = create_even_distribution(num_unique_items, num_slots)
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(item_distribution)]))
    return map_


def approx_mem_consumption(config):
    # in MB for sequence_length=100, batch_size=128, num_layers=2, num_units=2000
    # working for all lstm variants
    base_consumption = 4500
    consumption = base_consumption * \
        (config['num_units'] / 2000)**2 * \
        (config['num_layers'] / 2) * \
        (config['batch_size'] / 128) * \
        (config['sequence_length'] / 100)
    return consumption


def num_consequent_repeats(list_, idx):
    current = list_[idx]
    i = 1
    while list_[idx+i] == current:
        i += 1
    return i


def ceil(a):
    if int(a) != a:
        return int(a) + 1
    return int(a)


def split_experiment_config_into_separate_measurement_configs(config):
    seq_lens = make_list(config['sequence_length'])
    batch_sizes = make_list(config['batch_size'])
    num_layers = make_list(config['num_layers'])
    num_units = make_list(config['num_units'])
    combs = all_combs([seq_lens, batch_sizes, num_layers, num_units])
    configs = list()
    for comb in combs:
        conf = copy.deepcopy(config)
        conf['sequence_length'] = comb[0]
        conf['batch_size'] = comb[1]
        conf['num_layers'] = comb[2]
        conf['num_units'] = comb[3]
        configs.append(conf)
    if config['num_repeats'] == 1:
        return configs
    else:
        confs = list()
        for conf in configs:
            confs.extend([conf]*config['num_repeats'])
        return confs


def get_configs_run_in_parallel(configs, counter):
    current_config = configs[counter]
    max_num_in_parallel = current_config['memory_per_experiment'] / approx_mem_consumption(current_config)
    num_repeats = num_consequent_repeats(configs, counter)
    configs_to_process = configs[counter:counter+num_repeats]
    num_launches_required = ceil(num_repeats / max_num_in_parallel)
    distribution = create_even_distribution(num_launches_required, num_repeats)
    list_of_launches = list()
    pointer = 0
    for num_proc in distribution:
        list_of_launches.append(configs_to_process[pointer:pointer+num_proc])
        pointer += num_proc
    return list_of_launches, num_repeats


def make_list(candidate):
    if isinstance(candidate, list):
        l = candidate
    else:
        l = [candidate]
    return l

In [16]:
config = {
  "num_layers": [3, 4],
  "num_units": [1000, 2000],
  "sequence_length": [50, 100],
  "batch_size": [128],
  "save_path": "results/lstm_test",
  "lstm_type": "cudnn_stacked",
  "mode": "train",
  "num_steps": 500,
  "num_proc_in_pool": 5,
  "num_repeats": 10,
  "memory_per_experiment": 7500
}

In [17]:
configs = split_experiment_config_into_separate_measurement_configs(config)
launches = get_configs_run_in_parallel(configs, 0)
for idx, launch in enumerate(launches[0]):
    print(idx)
    for conf in launch:
        print(conf)
    print('\n')

0
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats'

In [18]:
print(approx_mem_consumption(configs[0]))

843.75
